<a href="https://colab.research.google.com/github/DirkStulgies/mlwtSportsPrediction/blob/main/playground/own_data_column_selection.ipynb"
 target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import packages.
import os
import signal
import tempfile
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorboard as tb

In [9]:
# Define parameters.
LOGS_PATH = '../logs'
DATA_PATH = '../data/own_data_preparation_dirk.csv'
RESULT_COLUMN = 'result_team1'
DUMMY_COLUMNS = ['team1', 'team2']
NORMAILZE_COLUMNS = ['points', 'squard',
    'average_age', 'average_market_value_in_euro', 'total_market_value_in_euro',
    'rank_last_season', 'points_last_season_all', 'points_last_season']
VALIDATION_SIZE = 0.2
LOSS_FUNCTIONS = ['categorical_hinge', 'hinge', 'huber', 
    'kullback_leibler_divergence', 'log_cosh', 'mean_absolute_error', 
    'mean_absolute_percentage_error', 'mean_squared_error', 'mean_squared_logarithmic_error',
    'sparse_categorical_crossentropy', 'squared_hinge', 'poisson']
OPTIMIZER_FUNCTIONS = ['Adadelta', 'Adagrad', 'Adam', 'Adamax', 'Ftrl', 'Nadam', 'RMSprop', 'SGD']
METRICS = 'sparse_categorical_accuracy'
EPOCHS = 200
BATCH_SIZE = 32
TITLE = 'optimizer_loss'

TENSORBOARD_PROCESS = 'tensorboard'
TENSORBOARD_SERVER = 'localhost'
TENSORBOARD_PORT = 6008

LOGS_PATH = os.path.join(LOGS_PATH, TITLE)

In [10]:
# Set up tensorboard.
%load_ext tensorboard
logs = Path(LOGS_PATH)
logs.mkdir(mode=0o777, parents=True, exist_ok=True)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
# Load the data set and drop rows with nan values.
data = pd.read_csv(DATA_PATH, delimiter=',', decimal='.')
data = data.dropna()

# Create the difference between team1 and team2 for each feature.
for column in NORMAILZE_COLUMNS:
    data[column] = data[column + '_team1'] - data[column + '_team2']

# Normalize column using the max value for each column.
for column in NORMAILZE_COLUMNS:
    max_value = data[column].max()
    data[column] = data[column].apply(lambda x: x / max_value)

# Get the wanted columns and create dummy columns for the teams.
data = data.loc[:, DUMMY_COLUMNS + NORMAILZE_COLUMNS + [RESULT_COLUMN]]
data = pd.get_dummies(data, columns=DUMMY_COLUMNS)

# Split the data set.
split_index = int(len(data) * VALIDATION_SIZE)
data = data.sample(frac=1)
data_train = data[split_index:]
data_valid = data[:split_index]

par_train = data_train.loc[:, data_train.columns != RESULT_COLUMN]
res_train = data_train.loc[:, [RESULT_COLUMN]]
par_valid = data_valid.loc[:, data_train.columns != RESULT_COLUMN]
res_valid = data_valid.loc[:, [RESULT_COLUMN]]

dummies = [key for key in par_train.keys() if 'team' in key]

data.head()

,points,squard,average_age,average_market_value_in_euro,total_market_value_in_euro,rank_last_season,points_last_season_all,points_last_season,result_team1,team1_1. FC Union Berlin,...,team2_Hertha Berlin,team2_Mainz,team2_RB Leipzig,team2_SC Freiburg,team2_SV Darmstadt 98,team2_Schalke 04,team2_TSG Hoffenheim,team2_VfB Stuttgart,team2_VfL Wolfsburg,team2_Werder Bremen
3460,-0.035714,0.470588,-0.205882,0.115168,0.306658,-0.133333,0.346154,0.090000,2,0,...,0,0,0,0,0,0,0,0,0,0
2436,0.107143,-0.058824,-0.088235,-0.052439,-0.072385,-0.133333,0.057692,0.012000,1,0,...,0,0,0,0,0,0,0,0,1,0
362,-0.250000,-0.294118,-0.029412,-0.005974,-0.022600,0.333333,-0.076923,-0.007273,1,0,...,0,0,0,0,0,0,0,0,0,0
26,0.053571,-0.470588,0.264706,0.270826,0.288596,-0.266667,0.538462,0.560000,2,0,...,0,1,0,0,0,0,0,0,0,0
373,-0.071429,0.000000,-0.029412,0.079655,0.097180,-0.733333,0.346154,0.030000,1,0,...,0,0,0,0,0,0,1,0,0,0


In [12]:
# Convert data frames to numpy arrays.
res_train = res_train.to_numpy()
res_valid = res_valid.to_numpy()

print(par_train.shape)
print(res_train.shape)
print(par_valid.shape)
print(res_valid.shape)

(4777, 54)
(4777, 1)
(1194, 54)
(1194, 1)


In [13]:
def defineModel(input_dim, optimizer, loss):
    # Define the model
    nodes = input_dim*2-1
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(nodes, activation='relu', input_dim=input_dim),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    # Compile the model.
    model.compile(loss=loss, optimizer=optimizer, metrics=[METRICS])

    return model


In [14]:
def runModel(model, x_train, x_valid, y_train, y_valid, title):
    # Define callback function for writing data for tensorBoard
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOGS_PATH, title), histogram_freq=1)

    # Run the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(x_valid, y_valid),
        callbacks=[tensorboard_callback],
        verbose=1
    )

    return history

In [15]:

# All columns with different optimizer and loss functions.
for optimizer in OPTIMIZER_FUNCTIONS:
    for loss in LOSS_FUNCTIONS:
        sel_train = par_train.to_numpy()
        sel_valid = par_valid.to_numpy()

        runModel(defineModel(sel_train.shape[1], optimizer, loss), sel_train, sel_valid, res_train, res_valid, optimizer+'_'+loss)



2022-01-08 19:47:40.898508: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-08 19:47:40.898546: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (u50sj3d10xnl): /proc/driver/nvidia/version does not exist
2022-01-08 19:47:40.898858: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
150/150 [==============================] - 1s 3ms/step - loss: 1.1022 - sparse_categorical_accuracy: 0.3634 - val_loss: 1.1009 - val_sparse_categorical_accuracy: 0.3610
Epoch 2/200
150/150 [==============================] - 0s 2ms/step - loss: 1.1059 - sparse_categorical_accuracy: 0.3500 - val_loss: 1.1003 - val_sparse_categorical_accuracy: 0.3643
Epoch 3/200
150/150 [==============================] - 0s 2ms/step - loss: 1.1015 - sparse_categorical_accuracy: 0.3563 - val_loss: 1.0998 - val_sparse_categorical_accuracy: 0.3643
Epoch 4/200
150/150 [==============================] - 0s 2ms/step - loss: 1.1017 - sparse_categorical_accuracy: 0.3601 - val_loss: 1.0992 - val_sparse_categorical_accuracy: 0.3660
Epoch 5/200
150/150 [==============================] - 0s 2ms/step - loss: 1.1019 - sparse_categorical_accuracy: 0.3529 - val_loss: 1.0987 - val_sparse_categorical_accuracy: 0.3685
Epoch 6/200
150/150 [==============================] - 0s 2ms/step - loss: 1.1011 - sparse_cate

In [16]:
# Kill the existing tensorboard process and delete the tensorflow temp folder. After this start a new tensorboard process.
try:    
    # Iterating through each instance of the process.
    for line in os.popen("ps ax | grep " + TENSORBOARD_PROCESS + " | grep -v grep"):
        fields = line.split()
            
        # Extracting Process ID from the output.
        pid = fields[0]
            
        # Terminating process.
        os.kill(int(pid), signal.SIGKILL)

    # Delete tensorboard temp folder.
    tb_temp_folder = os.path.join(tempfile.gettempdir(), '.tensorboard-info')
    os.system("rm -rf "+tb_temp_folder)
    print("Process Successfully terminated") 
except Exception as e:
    print(e)

%tensorboard --logdir $LOGS_PATH --host $TENSORBOARD_SERVER --port $TENSORBOARD_PORT

Process Successfully terminated
